Procesar JSON y convertirlo a texto

In [3]:
!pip install requests-cache


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 570.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00


In [13]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de empresas
empresas = [
    "Altitude Investment Management",
    "Alinga Capital",
    "Aurora Cannabis",
    "acreageholdings",
    "4 Front Ventures",
    "Barenberg Capital Partners",
    "Bayline Capital Partners",
    "Big Rock",
    "Bullpen Capital",
    "cannaangelsllc.com",
    "cannahedge.com",
    "Canalis Capital",
    "Cannabis Capital",
    "Canopy Rivers",
    "canopyboulder",
    "Charlottes Web Holdings",
    "casaverdecapital",
    "canopygrowth",
    "Compass Cannabis Clinic",
    "Constellation Brands",
    "Craft Ventures",
    "Cusp Capital",
    "Capital 8 Group",
    "First Trust Portafolio",
    "FundRx",
    "KES7 Capital",
    "Krauter Group",
    "KushCo Holdings",
    "LaMarch Capital",
    "Lerer Hippeau Ventures",
    "Mainline Investment Partners",
    "Mazakali",
    "Medmen",
    "Mentor Capital",
    "Merida Capital Partners",
    "Miracle Mile Advisors",
    "Montegra Capital",
    "muse capital",
    "Navy Capital",
    "Nosara Capital Growthinc",
    "Phyto Partners",
    "Privateer Holdings",
    "Rose Capital",
    "Salveo Capital",
    "Green Acre Capital",
    "Green Growth Investments",
    "Green Leaf Investments",
    "HIGH TIMES",
    "Initiative Capital",
    "Supreme Cannabis",
    "The Arcview Group",
    "Tress Capital",
    "High Tide",
    "Crowe",
    "Flower Hire",
    "Primary Growth Partners",
    "CHICAGO ATLANTIC GROUP",
    "GREEN SPACE ACCOUNTING",
    "Merida Capital Holdings",
    "Fluence",
    "Woodcrick Ventures"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=10)).strftime('%Y-%m-%d')

news_data = {}

for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    params = {
        'q': empresa,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        news_data[empresa] = []
        for article in articles:
            title = article['title']
            link = article['url']
            snippet = article['description']
            published_at = article['publishedAt']
            news_data[empresa].append(
                {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        news_data[empresa] = f"Error fetching data: {str(e)}"
        print(f"Error al buscar noticias para {empresa}: {str(e)}")

# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Altitude Investment Management
Encontradas 18 noticias para Altitude Investment Management
Buscando noticias para: Alinga Capital
Encontradas 0 noticias para Alinga Capital
Buscando noticias para: Aurora Cannabis
Encontradas 6 noticias para Aurora Cannabis
Buscando noticias para: acreageholdings
Encontradas 0 noticias para acreageholdings
Buscando noticias para: 4 Front Ventures
Encontradas 20 noticias para 4 Front Ventures
Buscando noticias para: Barenberg Capital Partners
Encontradas 0 noticias para Barenberg Capital Partners
Buscando noticias para: Bayline Capital Partners
Encontradas 0 noticias para Bayline Capital Partners
Buscando noticias para: Big Rock
Encontradas 100 noticias para Big Rock
Buscando noticias para: Bullpen Capital
Encontradas 4 noticias para Bullpen Capital
Buscando noticias para: cannaangelsllc.com
Encontradas 0 noticias para cannaangelsllc.com
Buscando noticias para: cannahedge.com
Encontradas 0 noticias para cannahedge.com
Buscando not

In [36]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for company, articles in news_data.items():
    if isinstance(articles, list):
        for article in articles:
            title = article.get('title', 'No title')
            snippet = article.get('snippet', 'No snippet')
            link = article.get('link', 'No link')
            extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Filtrar empresas sin noticias
filtered_elements = [element for element in extracted_elements if 'Title: No title' not in element]

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in filtered_elements]


# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.4, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [37]:

# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = [
    "¿De que se habla en las noticias recientes sobre la empresa?",
    "¿Cuáles son los 5 titulos con similitudes entre las noticias recientes sobre la empresa?",
    "¿Cuáles son los 5 titulos tienen noticias se repiten entre los diferentes artículos sobre la empresa?",
    "¿¿Cuáles son los 5  titulos de noticias mencionan los mismos eventos o desarrollos sobre la empresa?",
    "¿¿Cuáles son los 5  titulos de noticias discuten los mismos competidores en relación con la empresa?"
]

# Almacenar las respuestas en una lista
responses = []

for empresa in news_data:
    print(f"Analizando preguntas para: {empresa}")
    for j, query in enumerate(questions):
        full_query = f"{query} {empresa}"
        response = qa_chain({"query": full_query})
        responses.append({
            "empresa": empresa,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {empresa}")



Analizando preguntas para: Cannabis legalization
Pregunta 1/5 analizada para Cannabis legalization
Pregunta 2/5 analizada para Cannabis legalization
Pregunta 3/5 analizada para Cannabis legalization
Pregunta 4/5 analizada para Cannabis legalization
Pregunta 5/5 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/5 analizada para Medical marijuana
Pregunta 2/5 analizada para Medical marijuana
Pregunta 3/5 analizada para Medical marijuana
Pregunta 4/5 analizada para Medical marijuana
Pregunta 5/5 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/5 analizada para Cannabis industry
Pregunta 2/5 analizada para Cannabis industry
Pregunta 3/5 analizada para Cannabis industry
Pregunta 4/5 analizada para Cannabis industry
Pregunta 5/5 analizada para Cannabis industry
Analizando preguntas para: Cannabis investment
Pregunta 1/5 analizada para Cannabis investment
Pregunta 2/5 analizada para Cannabis investment
Pregunta

In [12]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(responses)

[         {         'empresa': 'Altitude Investment Management',
                    'query': '¿De que se habla en las noticias recientes sobre '
                             'la empresa?',
                    'response': 'Lo siento, no tengo información específica '
                                'sobre noticias recientes relacionadas con '
                                'Altitude Investment Management. Te '
                                'recomendaría buscar en fuentes de noticias '
                                'financieras o en el sitio web de la empresa '
                                'para obtener la información más actualizada.'},
          {         'empresa': 'Altitude Investment Management',
                    'query': '¿Cuáles son los titulos con similitudes entre '
                             'las noticias recientes sobre la empresa?',
                    'response': 'Lo siento, pero no tengo información sobre '
                                'noticias recientes r

In [24]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno

api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de temas relacionados con cannabis
temas = [
    "Cannabis legalization",
    "Medical marijuana",
    "Cannabis industry",
    "Cannabis investment",
    "Cannabis stocks",
    "Cannabis research",
    "Cannabis news",
    "Cannabis policy",
    "Cannabis business",
    "Cannabis culture",
    "Psychedelics research",
    "Psychedelics industry",
    "Psychedelics investment",
    "Psychedelics news",
    "Psychedelics policy",
    "Psychedelics business",
    "Psychedelics culture"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

news_data = {}

for tema in temas:
    print(f"Buscando noticias para: {tema}")
    params = {
        'q': tema,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        if articles:
            news_data[tema] = []
            for article in articles:
                title = article['title']
                link = article['url']
                snippet = article['description']
                published_at = article['publishedAt']
                news_data[tema].append(
                    {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
            print(f"Encontradas {len(articles)} noticias para {tema}")
        else:
            print(f"No se encontraron noticias para {tema}")
    except requests.exceptions.RequestException as e:
        print(f"Error al buscar noticias para {tema}: {str(e)}")

# Filtrar temas sin noticias
news_data = {tema: articles for tema, articles in news_data.items() if articles}

# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Cannabis legalization
Encontradas 20 noticias para Cannabis legalization
Buscando noticias para: Medical marijuana
Encontradas 57 noticias para Medical marijuana
Buscando noticias para: Cannabis industry
Encontradas 96 noticias para Cannabis industry
Buscando noticias para: Cannabis investment
Encontradas 33 noticias para Cannabis investment
Buscando noticias para: Cannabis stocks
Encontradas 30 noticias para Cannabis stocks
Buscando noticias para: Cannabis research
Encontradas 64 noticias para Cannabis research
Buscando noticias para: Cannabis news
Encontradas 100 noticias para Cannabis news
Buscando noticias para: Cannabis policy
Encontradas 52 noticias para Cannabis policy
Buscando noticias para: Cannabis business
Encontradas 100 noticias para Cannabis business
Buscando noticias para: Cannabis culture
Encontradas 18 noticias para Cannabis culture
Buscando noticias para: Psychedelics research
Encontradas 11 noticias para Psychedelics research
Buscando noticias

In [32]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for tema, articles in news_data.items():
    for article in articles:
        title = article.get('title', 'No title')
        snippet = article.get('snippet', 'No snippet')
        link = article.get('link', 'No link')
        extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in extracted_elements]

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [33]:
questions = [
    "¿Cuáles son los 5 titulos de las noticias con los temas más mencionados en las noticias recientes sobre el tema?",
    "¿Cuáles son los 5 titulos de las noticias sobre desarrollos importantes se destacan en las noticias recientes sobre el tema?",
    "¿Que personas o entidades son mencionadas con mayor frecuencia en las noticias sobre el tema?",
    "¿Qué impactos y consecuencias se mencionan con mayor frecuencia en las noticias recientes sobre el tema?",
    "¿Cuáles son los 5 titulos de las noticias que tienen patrones y similitudes se observan en los titulares y contenido de las noticias sobre el tema?"
]

# Almacenar las respuestas en una lista
responses = []

for tema in news_data:
    print(f"Analizando preguntas para: {tema}")
    for j, query in enumerate(questions):
        full_query = f"{query} {tema}"
        response = qa_chain({"query": full_query})
        responses.append({
            "tema": tema,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {tema}")



Analizando preguntas para: Cannabis legalization
Pregunta 1/5 analizada para Cannabis legalization
Pregunta 2/5 analizada para Cannabis legalization
Pregunta 3/5 analizada para Cannabis legalization
Pregunta 4/5 analizada para Cannabis legalization
Pregunta 5/5 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/5 analizada para Medical marijuana
Pregunta 2/5 analizada para Medical marijuana
Pregunta 3/5 analizada para Medical marijuana
Pregunta 4/5 analizada para Medical marijuana
Pregunta 5/5 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/5 analizada para Cannabis industry
Pregunta 2/5 analizada para Cannabis industry
Pregunta 3/5 analizada para Cannabis industry
Pregunta 4/5 analizada para Cannabis industry
Pregunta 5/5 analizada para Cannabis industry
Analizando preguntas para: Cannabis investment
Pregunta 1/5 analizada para Cannabis investment
Pregunta 2/5 analizada para Cannabis investment
Pregunta

In [34]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(responses)

[       {       'query': '¿Cuáles son los 5 titulos de las noticias con los '
                         'temas más mencionados en las noticias recientes '
                         'sobre el tema?',
                'response': '1. "Thai Legalization Advocates Rally as '
                            'Government Moves Toward Cannabis Ban"\n'
                            '2. "New Legislation Proposed for Cannabis '
                            'Legalization in Canada"\n'
                            '3. "US States Consider Cannabis Legalization '
                            'Amidst Public Debate"\n'
                            '4. "Global Trend Towards Cannabis Legalization '
                            'Sparks Policy Discussions"\n'
                            '5. "Medical Benefits of Cannabis Drive '
                            'Legalization Efforts Worldwide"',
                'tema': 'Cannabis legalization'},
        {       'query': '¿Cuáles son los 5 titulos de las noticias sobre '
      